## lidar 데이터를 point cloud CSV로

In [ ]:
import glob, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
lidar_data_dir = "C:\\Users\\acorn\\Documents\\Tank Challenge\\lidar_data"
points_dir = "C:\\Users\\acorn\\Documents\\Tank Challenge\\points"

COLS = ["x", "y", "z"]
COORD_STEP = 0.10  # 좌표 눈금간격
GRID_STEP = 10     # 격자로 나눌 때 x, z를 몇 단위로 나눌지 설정 (예: 10이면 0~10, 10~20...)
POINT_PREC = 1
get_file_name = lambda x, z: f"points-{x:03}-{z:03}.csv"

def get_points(x0:int, x1:int, z0:int, z1:int) -> pd.DataFrame:
    df = pd.DataFrame()
    for x in range(x0, x1, GRID_STEP):
        for z in range(z0, z1, GRID_STEP):
            file_path = os.path.join(points_dir, get_file_name(x, z))
            if not os.path.exists(file_path): continue
            df = pd.concat([df, pd.read_csv(file_path, header=None, names=COLS)], ignore_index=True)
    if df.empty: print('[WARN] get_points 결과가 없습니다.')
    return df


In [ ]:
# 해상도 제한, 중복 제거
def apply_coordstep_and_drop_dupl(df):
    df = df.copy()
    for col in COLS:
        df[col] = (df[col] // COORD_STEP).round().astype(int)
    df = df.drop_duplicates()
    for col in COLS:
        df[col] = df[col] * COORD_STEP
    # df[col] = df[col].apply(lambda x: int(x // COORD_STEP) * COORD_STEP)
    return df

# 점모으기: 라이다 데이터를 point cloud CSV로 내보냄.
def collect_points():
    # json 파일 안 쓰니까 다 삭제함.
    for f in glob.glob(os.path.join(lidar_data_dir, "*.json")):
        os.remove(f)

    updateds = set()

    for f in glob.glob(os.path.join(lidar_data_dir, "*.csv")):
        df = pd.read_csv(f)
        os.remove(f)

        df = df[df['isDetected'] == True]  # isDetected=True만
        df = df[COLS]                      # x,y,z 컬럼만
        df = df.dropna()

        if df.empty: continue

        df = apply_coordstep_and_drop_dupl(df)

        # 범위 제한 (우리 시뮬레이터 맵 범위)
        df = df[df['x'] >= 0]
        df = df[df['z'] >= 0]
        df = df[df['x'] < 300]
        df = df[df['z'] < 300]

        # 구간의 시작점 (내림)
        df['x_grid'] = (df['x'] // GRID_STEP).astype(int) * GRID_STEP
        df['z_grid'] = (df['z'] // GRID_STEP).astype(int) * GRID_STEP

        # x_grid, z_grid 별로 그룹핑하여 저장
        for (x_start, z_start), group in df.groupby(['x_grid', 'z_grid']):

            filename = get_file_name(x_start, z_start)
            updateds.add(filename)
            output_path = os.path.join(points_dir, filename)

            group[COLS].to_csv(output_path, index=False, header=False, mode="a")

    count = 0

    # 중복 제거
    for f in glob.glob(os.path.join(points_dir, "*.csv")):
        df = pd.read_csv(f, header=None, names=COLS)

        if f in updateds:
            df = apply_coordstep_and_drop_dupl(df)
            df.to_csv(f, index=False, header=False)

        count += len(df)

    print(f'updateds: {', '.join(updateds)}')
    print(f'count: {count}')

# collect_points()

In [ ]:
import json
from concurrent.futures import ThreadPoolExecutor
from flask import Flask, request, jsonify

import logging
log = logging.getLogger('werkzeug')
log.disabled = True

app = Flask(__name__)
executor = ThreadPoolExecutor(max_workers=4)

######################################
## x 값을 0, 10, 20, ... 바꿔가며 스캔
######################################

info = {}
target_xs = [x for x in range(0, 301, 10)]
target_x_idx = 0
target_x = lambda: target_xs[target_x_idx]
letsReset = False

# (/init) -> (/get_action) 맵끝에다다름 -> 일시정지, 점모으기 -> (/start) 시작하여 (/info) 리셋 (다음 target_x)


@app.route('/init', methods=['GET'])
def init():
    global letsReset
    letsReset = False

    x = target_x()
    print(f'x={x}')
    config = {
        "startMode": "start",

        "blStartX": x,
        "blStartY": 20,
        "blStartZ": 0,

        "rdStartX": 000, #Red Start Position
        "rdStartY": -50,
        "rdStartZ": 000,

        "trackingMode": True,
        "detactMode": False,
        "logMode": True,
        "enemyTracking": False,

        "saveSnapshot": False,
        "saveLog": False,
        "saveLidarData": True,
        "lux": 30000,
        "destoryObstaclesOnHit" : False
    }
    print("🛠️ INIT")
    return jsonify(config)


@app.route('/start', methods=['GET'])
def start():
    if letsReset:
        return jsonify({"control": "start"})
    else:
        return jsonify({"control": "pause"})


def collect_points_and_resume():
    global letsReset
    print('--- 점모으기 ---')
    collect_points()
    if target_x_idx >= len(target_xs):
        print('=== 끝 ===')
    else:
        print('--- 재개 ---')
        letsReset = True


@app.route('/info', methods=['POST'])
def api_info():
    global info, target_x_idx, letsReset

    if letsReset:
        return jsonify({"status": "success", "control": "reset"})

    info = request.get_json(force=True)

    # 맵의 끝에 다다르면 정지 후 점모으기 실행.
    if info['playerPos']['z'] >= 298:
        executor.submit(collect_points_and_resume)
        target_x_idx += 1
        return jsonify({"status": "success", "control": "pause"})

    return jsonify({"status": "success", "control": ""})


@app.route('/get_action', methods=['POST'])
def api_get_action():
    data = request.get_json(force=True)

    position = data.get("position", {})

    speed = 0.1

    # 차선 맞추기
    target_ad = target_x() - position.get("x")
    ad_value = ((target_ad - info['playerBodyX'] + 180) % 360 - 180) * 0.1 * speed

    return jsonify({
        "moveWS": {"command": "W", "weight": speed},
        "moveAD": {"command": "D" if ad_value > 0 else "A", "weight": ad_value if ad_value > 0 else -ad_value},
    })


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


In [ ]:
count = 0

# 중복 제거
for f in glob.glob(os.path.join(points_dir, "*.csv")):
    df = pd.read_csv(f, header=None, names=COLS)
    df = apply_coordstep_and_drop_dupl(df)
    df = df.sort_values(by=["x", "z", "y"], ascending=[True, True, True])
    df.to_csv(f, index=False, header=False)

    count += len(df)

print(f'count: {count}')

In [ ]:
points_dir = "C:\\Users\\acorn\\Documents\\Tank Challenge\\points-forest-1.0"

df = get_points(00, 300, 0, 300)

x = df["x"]
y = df["y"]
z = df["z"]

x_size = x.max() - x.min()
y_size = y.max() - y.min()
z_size = z.max() - z.min()
max_size = max(x_size, y_size, z_size)
x_size = 3 * x_size / max_size
y_size = 3 * y_size / max_size
z_size = 3 * z_size / max_size

fig = go.Figure(data=[go.Scatter3d(
    x=df["x"],
    y=df["z"],
    z=df["y"],
    mode='markers',
    marker=dict(
        size=5,
        color=df["y"],   # 높이에 따라 색상 변화
        colorscale='Viridis',
        opacity=1
    )
)])

fig.update_layout(
    width=1200,
    height=900,
    scene=dict(
        xaxis_title='x',
        yaxis_title='z',
        zaxis_title='y (height)',
        aspectmode='manual',
        aspectratio=dict(x=x_size, y=z_size, z=y_size),
    )
)

fig.show()